In [1]:
from jupyter_dash import JupyterDash 

In [2]:
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

In [3]:
df = pd.read_csv('../data/pm2_5.csv')

In [4]:
df.head()

,continent,continent_iso,code,country,year,pm2_5,gdp_per_capita,access_to_e_per,urban_pop
0,Africa,AF,ZWE,Zimbabwe,2010,23.231479,937.840338,38.782551,4262290
1,Africa,AF,ZWE,Zimbabwe,2011,23.074213,1082.615774,36.900002,4300463
2,Africa,AF,ZWE,Zimbabwe,2012,23.198097,1290.193956,44.000000,4355539
3,Africa,AF,ZWE,Zimbabwe,2013,23.111088,1408.367810,40.498375,4426387
4,Africa,AF,ZWE,Zimbabwe,2014,22.805950,1407.034293,32.299999,4503674


In [5]:
year_list = [i for i in range(2010, 2018, 1)]

In [6]:
year_list_2 = [i for i in range(2010, 2018, 1)]

In [7]:
country_list = df.country.unique()

In [8]:
app = dash.Dash(__name__)

In [ ]:
app.layout = html.Div([
    
       html.H1("PM2.5 Concentrations Dashboard"),
       html.Br(),
       html.H2("Air pollution info for the years 2010-2017 about PM2.5 Annual average Concentration levels globally", style = {'margin-top':'5px'}),
    
    html.Div(children = [dcc.Dropdown(id="slct_year",
                 options=[ {'label' : i, 'value': i} for i in year_list],
                 multi=False,
                 value=2010,
                 placeholder = 'Select Year ...',
                 ),
                         
                                              

   
    html.Br(),
    html.H2("Global Map of PM2.5 Concentrations"),
   

    dcc.Graph(id='pm2_5_map', figure={})

   

    
] ,style = {'margin-right':'50em'}),
   
    
    
      html.Div(children = [dcc.Dropdown(id='slct_country',
                                       options = [{'label' : i, 'value': i} for i in country_list],
                                       multi = False,
                                       value = 'Greece',
                                       placeholder = 'Select Country ...',),
                          html.Br(),
                          html.H2('Countries PM2.5 2010-2017'),
                          
                          dcc.Graph(id='line_plot_pm2_5', figure = {})], 
            style = {'margin-left':'50em', 'margin-top':'-30.45em'}),
    
    html.Br(),
    
     html.Div(children = [dcc.Dropdown(id='slct_year_2',
                                       options = [{'label' : i, 'value': i} for i in year_list_2],
                                       multi = False,
                                       value = 2010,
                                       placeholder = 'Select Year ...',),
                          html.Br(),
                          html.H2('Countries GDP Per Capita 2010-2017'),
                          
                          dcc.Graph(id='gdp_per', figure = {})], 
            style = {'margin-left': 'auto'}),
      




])
   
@app.callback(
     [Output(component_id='pm2_5_map', component_property='figure'),
      Output(component_id='line_plot_pm2_5', component_property='figure'),
      Output(component_id='gdp_per', component_property='figure')],
     [Input(component_id='slct_year', component_property='value'),
      Input(component_id='slct_country', component_property='value'),
      Input(component_id='slct_year_2', component_property='value')]
     
  
    )    

def update_my_graph(years_chosen, country_chosen, years_chosen_2):
    
    dff = df.copy()
    dff = dff[dff["year"] == years_chosen]
    dff_2 = df.copy()
    dff_2 = dff_2[dff_2['country'] == country_chosen]
    dff_3 = df.copy()
    dff_3 = dff_3[dff_3['year'] == years_chosen_2]
    

    
    fig_2 = px.line(data_frame=dff_2, x='year', y = 'pm2_5', color = 'country', symbol = 'country', labels={'country':'Country'})
    fig_2.update_xaxes(title_text = 'Year', title_font = dict(size = 14, family = 'Gilroy', color = '#008080')),
    fig_2.update_yaxes(title_text = 'PM2.5 Concentrations', title_font = dict(size = 14, family = 'Gilroy', color = '#008080'))
    fig_2.update_layout(template = 'plotly_dark')
    fig_2.update_xaxes(tickfont = dict(size = 14, family = 'Gilroy', color = '#008080'), showgrid = False)
    fig_2.update_yaxes(tickfont = dict(size = 14, family = 'Gilroy', color = '#008080'), showgrid = False)
    fig_2.update_layout(legend =dict(font=dict(family = 'Gilroy')))
    fig_2.update_layout(height = 450)
    fig_2.update_traces(line_color = 'rgba(24, 177, 217, 0.7)')
    fig_2.update_layout(legend = dict(font = dict( color = 'rgba(24, 177, 217, 0.7)')))
    
    fig = go.Figure(data=go.Choropleth(
    locations = dff['code'],
    z = dff['pm2_5'],
    text = dff['country'],
    colorscale = 'sunsetdark',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = 'μg/m3',
    colorbar_tickformat = '.1f',
    colorbar_title = 'PM2.5<br>μg/m3',
    colorbar_tickvals = [15,25,35,45,55,65,75,85,95],
    colorbar_ticktext = ['15μg/m3','25μg/m3','35μg/m3','45μg/m3','55μg/m3','65μg/m3','75μg/m3','85μg/m3','95μg/m3']))

    fig.update_layout(
    title_text='Global PM2.5 levels', title_x =0.5, title_y = 0.97, font = dict(family = 'Gilroy', color = '#008080', size = 11),
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    template = 'plotly_dark'
    )    
    
    
    
    
    fig_3 = go.Figure(data=go.Choropleth(
    locations = dff_3['code'],
    z = dff_3['gdp_per_capita'],
    text = dff_3['country'],
    colorscale = 'sunsetdark',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '$',
    colorbar_tickformat = '.1f',
    colorbar_title = 'GDP Per Capita<br>$',
    colorbar_tickvals = [5000,15000,25000,35000,45000,55000,65000,75000,85000,95000,105000],
    colorbar_ticktext = ['5000$','15000$','25000$','35000$','45000$','55000$','65000$','75000$','85000$','95000$','105000$']
))

    fig_3.update_layout(
    title_text='Global GDP Per Capita', title_x =0.5, title_y = 0.97, font = dict(family = 'Gilroy', color = '#008080', size = 11),
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    template = 'plotly_dark'
    )    

 
    return fig,fig_2,fig_3
   


if __name__ == '__main__':
    app.run_server(debug=False)
    

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [27/Mar/2023 22:22:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 22:22:09] "GET /assets/style.css?m=1679944850.4843276 HTTP/1.1" 304 -
127.0.0.1 - - [27/Mar/2023 22:22:09] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 22:22:09] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2023 22:22:10] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Mar/2023 22:22:10] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Mar/2023 22:22:10] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Mar/2023 22:22:10] "POST /_dash-update-component HTTP/1.1" 200 -
